In [1]:
import os, sys

notebook_dir = os.getcwd()

#add path
sys.path.append(os.path.abspath(os.path.join(notebook_dir, '..')))

In [2]:
from scripts.preprocess_data import *

In [3]:
preprocess_data("../telegram_messages.json")

Preprocessed data saved to preprocessed_data.csv


In [2]:
from scripts.CoNLL_formatting import *

In [3]:
# Read the data and save the subset_df in a variable
data = 'preprocessed_data.csv'
subset_df = read_subset_df(data)

# Call the label_data function to process and save the labeled data
label_data(subset_df)

         channel         sender                  timestamp  \
0  @ZemenExpress -1001307493052  2025-01-15 05:43:58+00:00   
1  @ZemenExpress -1001307493052  2025-01-15 05:43:58+00:00   
2  @ZemenExpress -1001307493052  2025-01-14 13:29:49+00:00   
3  @ZemenExpress -1001307493052  2025-01-14 13:29:49+00:00   
4  @ZemenExpress -1001307493052  2025-01-14 08:17:05+00:00   

                                             content  \
0                                                NaN   
1  💥💥..........................💥💥\n              ...   
2                                                NaN   
3  💥💥...................................💥💥\n\n📌Fe...   
4                                                NaN   

                                              tokens  
0                                                 []  
1  ['💥💥..........................💥💥', 'መልካም', 'በአ...  
2                                                 []  
3  ['💥💥...................................💥💥', '📌...  
4              

In [18]:
from scripts.load_conll_dataset import *
from scripts.tokenization_alignment import *
from scripts.training_and_tuning import *
from transformers import AutoTokenizer, AutoModelForTokenClassification
from sklearn.model_selection import train_test_split

In [19]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the variables for api
token = os.getenv('TOKEN')

In [20]:
# Load the connl data that we made earlier
file_path = 'labeled_data.conll'
sentences, labels = load_conll_dataset(file_path)


In [21]:
# execute_codes()
# sentences, labels = load_conll_dataset('labeled_data.conll')

# Define label mapping before loading the model
label_set = set(label for label_list in labels for label in label_list)
label_map = {label: i for i, label in enumerate(label_set)}


In [22]:

# Load the tokenizer and model
model_name = "rasyosef/bert-tiny-amharic"  # Change to "bert-tiny-amharic" or "afroxmlr" as needed
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=token)
model = AutoModelForTokenClassification.from_pretrained(model_name, use_auth_token=token)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at rasyosef/bert-tiny-amharic and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:

# Tokenize and align labels for the entire dataset
tokenized_dataset_full = tokenize_and_align_labels(sentences, labels, tokenizer, label_map)

# Split the dataset into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(sentences, labels, test_size=0.1)

# Tokenize and align labels for train and validation sets
train_tokenized = tokenize_and_align_labels(train_sentences, train_labels, tokenizer, label_map)
val_tokenized = tokenize_and_align_labels(val_sentences, val_labels, tokenizer, label_map)

# Create a dictionary for the tokenized dataset
tokenized_dataset = {
    'train': train_tokenized,
    'validation': val_tokenized
}

# Fine-tune the model
trainer = fine_tune_model(tokenized_dataset['train'], tokenized_dataset['validation'], model, tokenizer)

# Evaluate the model
eval_results = evaluate_model(trainer)
print(eval_results)

ValueError: With n_samples=0, test_size=0.1 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [16]:
# Tokenize and align labels
tokenized_dataset = tokenize_and_align_labels(sentences, labels, tokenizer, label_map)
print(tokenized_dataset)

# # Fine-tune the model
# trainer = fine_tune_model(tokenized_dataset['train'], tokenized_dataset['validation'], model, tokenizer)

# # Evaluate the model
# eval_results = evaluate_model(trainer)
# print(eval_results)

{'input_ids': [2, 3], 'token_type_ids': [0, 0], 'attention_mask': [1, 1], 'labels': []}


In [ ]:
# Save the fine-tuned model and tokenizer
save_model(trainer, tokenizer, "fine_tuned_ner_model")